In [44]:
import gymnasium
import flappy_bird_gymnasium
import pickle
import gc
import numpy as np
import pygame
import itertools
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from enum import IntEnum
from torchvision.transforms import Compose, ToTensor, Resize, Grayscale
from flappy_bird_gymnasium.envs.flappy_bird_env import FlappyBirdEnv
from flappy_bird_gymnasium.envs.flappy_bird_env import Actions
from flappy_bird_gymnasium.envs import utils
from flappy_bird_gymnasium.envs.lidar import LIDAR
from flappy_bird_gymnasium.envs.constants import (
    BACKGROUND_WIDTH,
    BASE_WIDTH,
    FILL_BACKGROUND_COLOR,
    LIDAR_MAX_DISTANCE,
    PIPE_HEIGHT,
    PIPE_VEL_X,
    PIPE_WIDTH,
    PLAYER_ACC_Y,
    PLAYER_FLAP_ACC,
    PLAYER_HEIGHT,
    PLAYER_MAX_VEL_Y,
    PLAYER_PRIVATE_ZONE,
    PLAYER_ROT_THR,
    PLAYER_VEL_ROT,
    PLAYER_WIDTH,
)
from typing import Dict, Optional, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties



def new_render(self):
    """Renders the next frame."""
    if self.render_mode == "rgb_array":
        self._draw_surface(show_score=False, show_rays=False)
        # Flip the image to retrieve a correct aspect
        return np.transpose(pygame.surfarray.array3d(self._surface), axes=(1, 0, 2))
    else:
        self._draw_surface(show_score=True, show_rays=False)
        if self._display is None:
            self._make_display()

        self._update_display()
        self._fps_clock.tick(self.metadata["render_fps"])

def new_step(
    self,
    action: Union[Actions, int],
) -> Tuple[np.ndarray, float, bool, Dict]:
    """Given an action, updates the game state.

    Args:
        action (Union[FlappyBirdLogic.Actions, int]): The action taken by
            the agent. Zero (0) means "do nothing" and one (1) means "flap".

    Returns:
        A tuple containing, respectively:

            * an observation (horizontal distance to the next pipe
                difference between the player's y position and the next hole's
                y position)
            * a reward (alive = +0.1, pipe = +1.0, dead = -1.0)
            * a status report (`True` if the game is over and `False`
                otherwise)
            * an info dictionary
    """
    """Given an action taken by the player, updates the game's state.

    Args:
        action (Union[FlappyBirdLogic.Actions, int]): The action taken by
            the player.

    Returns:
        `True` if the player is alive and `False` otherwise.
    """
    terminal = False
    reward = None

    self._sound_cache = None
    if action == Actions.FLAP:
        if self._player_y > -2 * PLAYER_HEIGHT:
            self._player_vel_y = PLAYER_FLAP_ACC
            self._player_flapped = True
            self._sound_cache = "wing"

    # check for score
    player_mid_pos = self._player_x + PLAYER_WIDTH / 2
    for pipe in self._upper_pipes:
        pipe_mid_pos = pipe["x"] + PIPE_WIDTH / 2
        if pipe_mid_pos <= player_mid_pos < pipe_mid_pos + 4:
            self._score += 1
            reward = 1.0  # reward for passed pipe
            self._sound_cache = "point"

    # player_index base_x change
    if (self._loop_iter + 1) % 3 == 0:
        self._player_idx = next(self._player_idx_gen)

    self._loop_iter = (self._loop_iter + 1) % 30
    self._ground["x"] = -((-self._ground["x"] + 100) % self._base_shift)

    # rotate the player
    if self._player_rot > -90:
        self._player_rot -= PLAYER_VEL_ROT

    # player's movement
    if self._player_vel_y < PLAYER_MAX_VEL_Y and not self._player_flapped:
        self._player_vel_y += PLAYER_ACC_Y

    if self._player_flapped:
        self._player_flapped = False

        # more rotation to cover the threshold
        # (calculated in visible rotation)
        self._player_rot = 45

    self._player_y += min(
        self._player_vel_y, self._ground["y"] - self._player_y - PLAYER_HEIGHT
    )

    # move pipes to left
    for up_pipe, low_pipe in zip(self._upper_pipes, self._lower_pipes):
        up_pipe["x"] += PIPE_VEL_X
        low_pipe["x"] += PIPE_VEL_X

        # it is out of the screen
        if up_pipe["x"] < -PIPE_WIDTH:
            new_up_pipe, new_low_pipe = self._get_random_pipe()
            up_pipe["x"] = new_up_pipe["x"]
            up_pipe["y"] = new_up_pipe["y"]
            low_pipe["x"] = new_low_pipe["x"]
            low_pipe["y"] = new_low_pipe["y"]

    if self.render_mode == "human":
        self.render()

    obs, reward_private_zone = self._get_observation()
    if reward is None:
        if reward_private_zone is not None:
            reward = float(reward_private_zone)
        else:
            reward = 0.1  # reward for staying alive

    # check
    if self._debug and self._use_lidar:
        # sort pipes by the distance between pipe and agent
        up_pipe = sorted(
            self._upper_pipes,
            key=lambda x: np.sqrt(
                (self._player_x - x["x"]) ** 2
                + (self._player_y - (x["y"] + PIPE_HEIGHT)) ** 2
            ),
        )[0]
        # find ray closest to the obstacle
        min_index = np.argmin(obs)
        min_value = obs[min_index] * LIDAR_MAX_DISTANCE
        # mean approach to the obstacle
        if "pipe_mean_value" in self._statistics:
            self._statistics["pipe_mean_value"] = self._statistics[
                "pipe_mean_value"
            ] * 0.99 + min_value * (1 - 0.99)
        else:
            self._statistics["pipe_mean_value"] = min_value

        # Nearest to the pipe
        if "pipe_min_value" in self._statistics:
            if min_value < self._statistics["pipe_min_value"]:
                self._statistics["pipe_min_value"] = min_value
                self._statistics["pipe_min_index"] = min_index
        else:
            self._statistics["pipe_min_value"] = min_value
            self._statistics["pipe_min_index"] = min_index

        # Nearest to the ground
        diff = np.abs(self._player_y - self._ground["y"])
        if "ground_min_value" in self._statistics:
            if diff < self._statistics["ground_min_value"]:
                self._statistics["ground_min_value"] = diff
        else:
            self._statistics["ground_min_value"] = diff

    # agent touch the top of the screen as punishment
    if self._player_y < 0:
        reward = -0.5

    # check for crash
    if self._check_crash():
        self._sound_cache = "hit"
        reward = -1.0  # reward for dying
        terminal = True
        self._player_vel_y = 0
        if self._debug and self._use_lidar:
            if ((self._player_x + PLAYER_WIDTH) - up_pipe["x"]) > (0 + 5) and (
                self._player_x - up_pipe["x"]
            ) < PIPE_WIDTH:
                print("BETWEEN PIPES")
            elif ((self._player_x + PLAYER_WIDTH) - up_pipe["x"]) < (0 + 5):
                print("IN FRONT OF")
            print(
                f"obs: [{self._statistics['pipe_min_index']},"
                f"{self._statistics['pipe_min_value']},"
                f"{self._statistics['pipe_mean_value']}],"
                f"Ground: {self._statistics['ground_min_value']}"
            )

    info = {"score": self._score}

    return (
        obs,
        np.float32(reward),
        terminal,
        (self._score_limit is not None) and (self._score >= self._score_limit),
        info,
    )


FlappyBirdEnv.render = new_render
FlappyBirdEnv.step = new_step

In [50]:
###############
######DQN######
###############

class DQN(nn.Module):
    def __init__(self, input_dim, action_space):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # First hidden layer
        self.fc2 = nn.Linear(128, 128)   
        self.fc3 = nn.Linear(128, 128)      # Second hidden layer
        self.fc4 = nn.Linear(128, action_space)  # Output layer for Q-values

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.fc4(x)



# Replay Memory to store experiences
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


# DQN Agent Class
class DQNAgent:
    def __init__(self, 
                 env,
                 hyper ={
                          "learning_rate": 0.001,
                          "discount_factor" : 0.99,
                          "epsilon" : 1.0,
                          "epsilon_decay" :0.999,
                          "epsilon_min" : 0.01,
                          "batch_size" : 64,
                          "memory_size" : 10000,
                          "episodes" : 100000,
                          "target_update_freq" : 10
                        }
                 ):
        # Environment
        self.env = env
        self.state_dim = env.observation_space.shape[0]  # First dimension of observation
        self.action_space = env.action_space.n


        # Hyperparameters
        self.learning_rate = hyper["learning_rate"]
        self.discount_factor = hyper["discount_factor"]
        self.epsilon = hyper["epsilon"]
        self.epsilon_decay = hyper["epsilon_decay"]
        self.epsilon_min = hyper["epsilon_min"]
        self.batch_size = hyper["batch_size"]
        self.memory_size = hyper["memory_size"]
        self.episodes = hyper["episodes"]
        self.target_update_freq = hyper["target_update_freq"]
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(self.device)


        # Initialize policy and target networks
        self.policy_net = DQN(self.state_dim, self.action_space).to(self.device)
        self.target_net = DQN(self.state_dim, self.action_space).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        # Optimizer and Replay Memory
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.memory = ReplayMemory(self.memory_size)

    def select_action(self, state, testing=False):
        """Epsilon-greedy action selection."""
        if not testing and random.random() < self.epsilon:
            return random.randint(0, self.action_space - 1)  # Random action
        else:
            with torch.no_grad():
                state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
                return self.policy_net(state).argmax(dim=1).item()

    def optimize_model(self):
        """Sample a batch from memory and optimize the policy network."""
        if len(self.memory) < self.batch_size:
            return

        batch = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert to tensors and move to device
        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.long).unsqueeze(1).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(self.device)

        # Compute Q-values and targets
        q_values = self.policy_net(states).gather(1, actions)
        next_q_values = self.target_net(next_states).max(1, keepdim=True)[0]
        targets = rewards + (self.discount_factor * next_q_values * (1 - dones))

        # Loss and backpropagation
        loss = nn.MSELoss()(q_values, targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def train(self):
        res=[]
        """Train the agent."""
        for episode in range(self.episodes):
            state, _ = self.env.reset()
            done = False
            total_reward = 0.0
            while not done:
                # Select and execute action
                action = self.select_action(state)
                next_state, reward, done, _, _ = self.env.step(action)
                self.memory.push(state, action, reward, next_state, done)
                state = next_state
                total_reward += reward

                # Optimize model
                self.optimize_model()

            # Update target network periodically
            if episode % self.target_update_freq == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict())

            # Decay epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            res.append(total_reward)
            print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {self.epsilon:.4f}")

        self.env.close()
        print("Training complete!")

        return res

    def test(self, num_episodes=10,render="human"):
        """Test the trained policy with real-time rendering."""
        print("\nTesting the trained policy...\n")
        self.epsilon = 0.0  # Disable exploration
        test_env = gymnasium.make("FlappyBird-v0", render_mode=render,use_lidar=True)  # Render in "human" mode 
        total_rewards = []

        for episode in range(num_episodes):
            state, _ = test_env.reset()
            done = False
            total_reward = 0.0

            while not done:
                action = self.select_action(state, testing=True)
                next_state, reward, done, _, _ = test_env.step(action)
                state = next_state
                total_reward += reward

            total_rewards.append(total_reward)
            print(f"Test Episode: {episode + 1}, Total Reward: {total_reward}")

        avg_reward = np.mean(total_rewards)
        print(f"\nAverage Reward over {num_episodes} Test Episodes: {avg_reward}")
        test_env.close()
        return total_rewards

In [52]:


# Initialize environment
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array", use_lidar=True)

# Hyperparameters
hyper = {
    "learning_rate": [0.001, 0.005],
    "discount_factor": 0.99,
    "epsilon": 1.0,
    "epsilon_decay": [0.995, 0.999],
    "epsilon_min": 0.01,
    "batch_size": 64,
    "memory_size": 10000,
    "episodes": 20000,
    "target_update_freq": [10, 20]
}

param_combinations = itertools.product(
    hyper["learning_rate"], 
    hyper["epsilon_decay"], 
    hyper["target_update_freq"]
)

# Initialize result dictionaries
exp_res = {}
test_res = {}

# Iterate through parameter combinations
for lr, epsilon_decay, target_update_freq in param_combinations:
    current_hyperparams = {
        "learning_rate": lr,
        "discount_factor": hyper["discount_factor"],
        "epsilon": hyper["epsilon"],
        "epsilon_decay": epsilon_decay,
        "epsilon_min": hyper["epsilon_min"],
        "batch_size": hyper["batch_size"],
        "memory_size": hyper["memory_size"],
        "episodes": hyper["episodes"],
        "target_update_freq": target_update_freq
    }
    
    try:
        # Train the agent
        agent = DQNAgent(env, current_hyperparams)
        exp_key = f"lr={lr}_decay={epsilon_decay}_freq={target_update_freq}"
        exp_res[exp_key] = agent.train()
        
        # Test the agent
        test_res[exp_key] = agent.test(num_episodes=5000, render=None)
        
        # Save intermediate results
        with open("../exp_data/DQN_tr.pkl", "wb") as f:
            pickle.dump(exp_res, f)
        with open("../exp_data/DQN_test.pkl", "wb") as f:
            pickle.dump(test_res, f)
        
        print(f"Finished training and testing for: {exp_key}")
    
    finally:
        # Free resources
        del agent
        gc.collect()
        env.close()


with open("../exp_data/DQN_tr.pkl", "rb") as f:
    exp_res = pickle.load(f)
with open("../exp_data/DQN_test.pkl", "rb") as f:
    test_res = pickle.load(f)

print("All parameter combinations processed.")


cuda
Episode: 1, Total Reward: -8.100000381469727, Epsilon: 0.9950
Episode: 2, Total Reward: -8.100000381469727, Epsilon: 0.9900
Training complete!

Testing the trained policy...

Test Episode: 1, Total Reward: -9.300000190734863
Test Episode: 2, Total Reward: -9.300000190734863
Test Episode: 3, Total Reward: -9.300000190734863
Test Episode: 4, Total Reward: -9.300000190734863
Test Episode: 5, Total Reward: -9.300000190734863

Average Reward over 5 Test Episodes: -9.300000190734863
Finished training and testing for: lr=0.001_decay=0.995_freq=10
cuda
Episode: 1, Total Reward: -8.100000381469727, Epsilon: 0.9950
Episode: 2, Total Reward: -8.100000381469727, Epsilon: 0.9900
Training complete!

Testing the trained policy...

Test Episode: 1, Total Reward: -0.9000015258789062
Test Episode: 2, Total Reward: -0.9000015258789062
Test Episode: 3, Total Reward: -0.9000015258789062
Test Episode: 4, Total Reward: -0.9000015258789062
Test Episode: 5, Total Reward: -1.5000016689300537

Average Rewar